<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="3em" style="font-weight:bold;color:#3f8dbf;">  </font><br>
    
### 주요 질문(탐색하고자 하는 질문 리스트)
    
- **일별, 주별, 월별 판매 트렌드는?**
- 어떤 카테고리가 가장 많은 상품이 팔렸을까?
- 평균 배송시간은?
</div>

In [1]:
import pandas as pd

customers = pd.read_csv('00_data/olist_customers_dataset.csv', encoding = 'utf-8-sig')
geolocations = pd.read_csv('00_data/olist_geolocation_dataset.csv', encoding = 'utf-8-sig')
items = pd.read_csv('00_data/olist_order_items_dataset.csv', encoding = 'utf-8-sig')
payments = pd.read_csv('00_data/olist_order_payments_dataset.csv', encoding = 'utf-8-sig')
reviews = pd.read_csv('00_data/olist_order_reviews_dataset.csv', encoding = 'utf-8-sig')
orders = pd.read_csv('00_data/olist_orders_dataset.csv', encoding = 'utf-8-sig')
sellers = pd.read_csv('00_data/olist_sellers_dataset.csv', encoding = 'utf-8-sig')
products = pd.read_csv('00_data/olist_products_dataset.csv', encoding = 'utf-8-sig')

In [3]:
payments_copy = payments.groupby('order_id').sum()

orders_payments = pd.merge(orders, payments_copy, on = 'order_id')
orders_payments['order_purchase_timestamp'] = pd.to_datetime(orders_payments['order_purchase_timestamp'], format = '%Y-%m-%d %H:%M:%S', errors = 'raise')

drop_canceled = orders_payments.dropna()[orders_payments['order_status'] == 'canceled'].index.to_list()
for drops in drop_canceled:
    orders_payments = orders_payments.dropna().drop([drops])

orders_payments_delivered = orders_payments[['order_purchase_timestamp','payment_value']].copy()

<ipython-input-3-b5d1261469c6>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  drop_canceled = orders_payments.dropna()[orders_payments['order_status'] == 'canceled'].index.to_list()


#### - dt

In [4]:
orders_payments_delivered['year'] = orders_payments_delivered['order_purchase_timestamp'].dt.year
orders_payments_delivered['quarter'] = orders_payments_delivered['order_purchase_timestamp'].dt.quarter
orders_payments_delivered['month'] = orders_payments_delivered['order_purchase_timestamp'].dt.month
orders_payments_delivered['monthday'] = orders_payments_delivered['order_purchase_timestamp'].dt.day
orders_payments_delivered['weekday'] = orders_payments_delivered['order_purchase_timestamp'].dt.weekday
orders_payments_delivered['hour'] = orders_payments_delivered['order_purchase_timestamp'].dt.hour
orders_payments_delivered['minute'] = orders_payments_delivered['order_purchase_timestamp'].dt.minute

orders_payments_delivered.head()

,order_purchase_timestamp,payment_value,year,quarter,month,monthday,weekday,hour,minute
0,2017-10-02 10:56:33,38.71,2017,4,10,2,0,10,56
1,2018-07-24 20:41:37,141.46,2018,3,7,24,1,20,41
2,2018-08-08 08:38:49,179.12,2018,3,8,8,2,8,38
3,2017-11-18 19:28:06,72.20,2017,4,11,18,5,19,28
4,2018-02-13 21:18:39,28.62,2018,1,2,13,1,21,18


#### -요일별

In [5]:
orders_payments_weekday = orders_payments_delivered[['weekday', 'payment_value']]
orders_payments_weekday.head()

,weekday,payment_value
0,0,38.71
1,1,141.46
2,2,179.12
3,5,72.20
4,1,28.62


In [6]:
def func(row):
    if row['weekday'] == 0:
        row['weekday'] = "Sun"
    elif row['weekday'] == 1:
        row['weekday'] = "Mon"
    elif row['weekday'] == 2:
        row['weekday'] = "Tue"
    elif row['weekday'] == 3:
        row['weekday'] = "Wed"
    elif row['weekday'] == 4:
        row['weekday'] = "Thu"
    elif row['weekday'] == 5:
        row['weekday'] = "Fri"
    elif row['weekday'] == 6:
        row['weekday'] = "Sat"
        
    return row
    #까먹지말고!
    
orders_payments_weekday = orders_payments_weekday.apply(func, axis=1).groupby('weekday').sum()
orders_payments_weekday

,payment_value
weekday,
Fri,1704838.47
Mon,2473930.77
Sat,1807350.89
Sun,2530591.86
Thu,2222157.43
Tue,2396215.34
Wed,2283849.51


---
- 한번에 진행했더니 요일의 알파벳 순으로 정렬

In [7]:
orders_payments_weekday.sort_index(level=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])

,payment_value
weekday,
Fri,1704838.47
Mon,2473930.77
Sat,1807350.89
Sun,2530591.86
Thu,2222157.43
Tue,2396215.34
Wed,2283849.51


- 실패

---
- 어차피 이렇게된거 sort행을 추가하자

In [8]:
type(orders_payments_delivered['order_purchase_timestamp'].dt.year)

pandas.core.series.Series

그냥 리스트 형태로

In [9]:
orders_payments_weekday['sort'] = [5,1,6,7,4,2,3]
orders_payments_weekday

,payment_value,sort
weekday,,
Fri,1704838.47,5
Mon,2473930.77,1
Sat,1807350.89,6
Sun,2530591.86,7
Thu,2222157.43,4
Tue,2396215.34,2
Wed,2283849.51,3


In [10]:
orders_payments_weekday = orders_payments_weekday.sort_values(by='sort')
orders_payments_weekday

,payment_value,sort
weekday,,
Mon,2473930.77,1
Tue,2396215.34,2
Wed,2283849.51,3
Thu,2222157.43,4
Fri,1704838.47,5
Sat,1807350.89,6
Sun,2530591.86,7


In [11]:
del orders_payments_weekday['sort']
orders_payments_weekday

,payment_value
weekday,
Mon,2473930.77
Tue,2396215.34
Wed,2283849.51
Thu,2222157.43
Fri,1704838.47
Sat,1807350.89
Sun,2530591.86


In [12]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

orders_payments_weekday.iplot(kind = 'bar')

#### -분기별

In [13]:
orders_payments_quarter = orders_payments_delivered[['quarter','payment_value']]
orders_payments_quarter.head()

,quarter,payment_value
0,4,38.71
1,3,141.46
2,3,179.12
3,4,72.20
4,1,28.62


#### -시간대

In [14]:
orders_payments_hour = orders_payments_delivered[['hour', 'payment_value']]
orders_payments_hour.head()

,hour,payment_value
0,10,38.71
1,20,141.46
2,8,179.12
3,19,72.20
4,21,28.62


In [15]:
orders_payments_hour = orders_payments_hour.groupby('hour').sum()

orders_payments_hour.iplot(kind = 'bar')

In [16]:
layout = {
    'xaxis':{
        'showticklabels':True,
        'dtick':1
    }
}
    
orders_payments_hour.iplot(kind = 'bar', layout=layout)

#### -요일/시간 상관관계

In [17]:
orders_payments_weekday_hour = orders_payments_delivered[['weekday', 'hour','payment_value']].copy()
orders_payments_weekday_hour = orders_payments_weekday_hour.groupby(['weekday', 'hour']).sum()
#                                                                   [    멀티인덱스  ]
orders_payments_weekday_hour

payment_value
weekday hour               
0       0          46807.42
        1          17734.77
        2           7462.56
        3           6442.62
        4           2138.02
...                     ...
6       19        140127.39
        20        150543.96
        21        135431.31
        22        127866.74
        23         82991.95

[168 rows x 1 columns]

In [18]:
orders_payments_delivered.head()

,order_purchase_timestamp,payment_value,year,quarter,month,monthday,weekday,hour,minute
0,2017-10-02 10:56:33,38.71,2017,4,10,2,0,10,56
1,2018-07-24 20:41:37,141.46,2018,3,7,24,1,20,41
2,2018-08-08 08:38:49,179.12,2018,3,8,8,2,8,38
3,2017-11-18 19:28:06,72.20,2017,4,11,18,5,19,28
4,2018-02-13 21:18:39,28.62,2018,1,2,13,1,21,18


In [19]:
orders_payments_delivered.columns

Index(['order_purchase_timestamp', 'payment_value', 'year', 'quarter', 'month',
       'monthday', 'weekday', 'hour', 'minute'],
      dtype='object')

In [20]:
orders_payments_weekday_hour = orders_payments_weekday_hour.reset_index()
orders_payments_weekday_hour

,weekday,hour,payment_value
0,0,0,46807.42
1,0,1,17734.77
2,0,2,7462.56
3,0,3,6442.62
4,0,4,2138.02
...,...,...,...
163,6,19,140127.39
164,6,20,150543.96
165,6,21,135431.31
166,6,22,127866.74


In [21]:
orders_payments_weekday_hour = orders_payments_weekday_hour.apply(func,axis=1)
orders_payments_weekday_hour['weekday'] = pd.Categorical(orders_payments_weekday_hour['weekday'], categories=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'], ordered=True)
# 이겨였노
orders_payments_weekday_hour = orders_payments_weekday_hour.sort_values(by=['weekday','hour'])
orders_payments_weekday_hour

,weekday,hour,payment_value
24,Mon,0.0,46897.54
25,Mon,1.0,23357.21
26,Mon,2.0,9775.06
27,Mon,3.0,5766.02
28,Mon,4.0,4011.98
...,...,...,...
19,Sun,19.0,155145.05
20,Sun,20.0,162640.98
21,Sun,21.0,174651.17
22,Sun,22.0,165825.81


In [23]:
orders_payments_weekday_hour.iplot(kind='heatmap',x='hour',y='weekday',z='payment_value',colorscale='Blues')

In [24]:
layout= {
    'title':{
        'text': '<b>Heatmap of Turnover over the Hour by Day</b>',
        'x':0.5,
        'y':0.9,
        'font':{
            'size':15
        }
    },
    'xaxis':{
        'title':'Hour',
        'showticklabels':True,
        'dtick':1
    },
    'yaxis':{
        'title':'Day'
    },
    'template': 'plotly_white'
}

orders_payments_weekday_hour.iplot(kind='heatmap',x='hour',y='weekday',z='payment_value',colorscale='Reds', layout=layout)